In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
!pip install streamlit
import streamlit as st
import requests
import json
from PIL import Image
from io import BytesIO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install piexif


In [ ]:
import os
import requests
import piexif
from PIL import Image
from io import BytesIO
from ultralytics import YOLO

# Define model path
MODEL_PATH = "/content/best.pt"

# Ensure model exists
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model file not found: {MODEL_PATH}. Check training output.")

# Load trained YOLOv8 model
model = YOLO(MODEL_PATH)

# Image URL (Replace with a different URL if needed)
image_url = 'https://media.istockphoto.com/id/183851840/photo/bad-repair-pothole-in-road-t-junction-suffers-frost-damage.jpg?s=612x612&w=0&k=20&c=C6x40SIitvOnljrXy-1AZcZ16k3rhmkqnXEDVz-ifZ0='

# Step 1: Download and save the image
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))
image_path = 'downloaded_image.jpg'
img.save(image_path)

# Step 2: Extract GPS Coordinates from Image Metadata
def extract_gps_coordinates(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()

    if exif_data is not None:
        gps_info = exif_data.get(34853)  # GPSInfo tag
        if gps_info:
            lat = gps_info.get(2)
            lon = gps_info.get(4)
            if lat and lon:
                lat_deg = lat[0] + lat[1] / 60.0 + lat[2] / 3600.0
                lon_deg = lon[0] + lon[1] / 60.0 + lon[2] / 3600.0
                return lat_deg, lon_deg
    return None, None

lat, lon = extract_gps_coordinates(image_path)

# Step 3: Perform detection using YOLOv8
results = model.predict(source=image_path, stream=False, save=True)

# Class names (ensure they match your dataset)
class_names = ['D00', 'D01', 'D10', 'D11', 'D20', 'D40', 'D43', 'D44']

# Store confidence values
confidences = []

# Process results
for result in results:
    boxes = result.boxes  # Get bounding boxes
    if len(boxes) == 0:
        print("No road damage detected.")
    else:
        for box in boxes:
            class_id = int(box.cls)  # Class index
            confidence = float(box.conf) * 100  # Convert confidence to percentage
            confidences.append(confidence)
            class_name = class_names[class_id]
            print(f"Detected: {class_name} with {confidence:.2f}% confidence")

# Calculate damage severity
damage_severity = sum(confidences) / len(confidences) if confidences else 0
print(f"\nThe road has {damage_severity:.2f}% damage severity.")

# Step 4: Fetch real-time weather, traffic, and natural calamity data (Placeholder API URLs)
def fetch_weather_and_traffic(lat, lon):
    if lat is None or lon is None:
        return None, None, None

    weather_api_url = f"https://api.weather.com/v3/wx/conditions/current?geocode={lat},{lon}&format=json"
    traffic_api_url = f"https://api.traffic.com/v1/conditions?lat={lat}&lon={lon}"
    calamity_api_url = f"https://api.disaster.com/v1/events?lat={lat}&lon={lon}"  # Placeholder API

    weather_data = requests.get(weather_api_url).json()
    traffic_data = requests.get(traffic_api_url).json()
    calamity_data = requests.get(calamity_api_url).json()

    return weather_data, traffic_data, calamity_data

weather_data, traffic_data, calamity_data = fetch_weather_and_traffic(lat, lon)

# Step 5: Calculate risk score
def calculate_risk_score(damage_severity, weather_data, traffic_data, calamity_data):
    traffic_factor = traffic_data.get("congestion", 1) if traffic_data else 1
    weather_factors = (
        weather_data.get("precipitation", 0) +
        weather_data.get("temperature", 20) / 100 +
        weather_data.get("humidity", 50) / 100 +
        weather_data.get("wind_speed", 10) / 50 +
        weather_data.get("uv_index", 5) / 10 +
        weather_data.get("snow", 0) / 10
    ) if weather_data else 1

    calamity_factor = (
        calamity_data.get("earthquake", 0) * 2 +
        calamity_data.get("flood", 0) * 2 +
        calamity_data.get("hurricane", 0) * 2 +
        calamity_data.get("landslide", 0) * 2 +
        calamity_data.get("tsunami", 0) * 2
    ) if calamity_data else 1

    risk_score = (damage_severity * 0.4) + (traffic_factor * 0.3) + (weather_factors * 0.2) + (calamity_factor * 0.1)
    return min(risk_score, 100)

risk_score = calculate_risk_score(damage_severity, weather_data, traffic_data, calamity_data)

# Step 6: Estimate repair cost
def estimate_repair_cost(damage_severity, area=50):  # Assuming 50 sq. meters per image
    base_cost_per_sqm = 100  # Example cost per square meter
    return damage_severity * area * base_cost_per_sqm / 100

repair_cost = estimate_repair_cost(damage_severity)

# Step 7: Estimate next maintenance check
def estimate_maintenance_interval(risk_score, damage_severity):
    if risk_score > 90 or damage_severity > 80:
        return "❌ Immediate maintenance required!"
    elif risk_score > 80 or damage_severity > 60:
        return "⚠️ Maintenance in 3 months"
    elif risk_score > 60 or damage_severity > 40:
        return "🔧 Maintenance in 6 months"
    elif risk_score > 40:
        return "🛠️ Maintenance in 1 year"
    else:
        return "✅ Maintenance in 2+ years"

maintenance_interval = estimate_maintenance_interval(risk_score, damage_severity)

# Step 8: Display final maintenance report
def display_maintenance_report(damage_severity, risk_score, repair_cost, maintenance_interval):
    return (f"📌 Damage Severity: {damage_severity:.2f}%\n"
            f"⚠️ Risk Level: {'Critical ❌' if risk_score > 90 else 'Moderate ⚠️' if risk_score > 60 else 'Low ✅'}\n"
            f"💰 Estimated Repair Cost: ${repair_cost:.2f}\n"
            f"🛠️ Next Maintenance Check: {maintenance_interval}")

print("\n🚧 **MAINTENANCE REPORT** 🚧")
print(display_maintenance_report(damage_severity, risk_score, repair_cost, maintenance_interval))



image 1/1 /content/downloaded_image.jpg: 448x640 1 D01, 1 D11, 1215.8ms
Speed: 9.4ms preprocess, 1215.8ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict3
Detected: D01 with 43.27% confidence
Detected: D11 with 42.88% confidence

The road has 43.08% damage severity.

🚧 **MAINTENANCE REPORT** 🚧
📌 Damage Severity: 43.08%
⚠️ Risk Level: Low ✅
💰 Estimated Repair Cost: $2153.83
🛠️ Next Maintenance Check: 🔧 Maintenance in 6 months
